# 대장암 인원
- 21,773 명
- T200 & T400 에서 없는 피험자 걸러져서 21,741명으로 단축
- 재발 194명 
- 재발X 21,547명

# 필요 라이브러리

In [1]:
import warnings
import os
import pandas as pd

warnings.filterwarnings('ignore')

In [2]:
print(os.getcwd()) # 현재 경로
# print(os.listdir(os.getcwd())) # 현재 경로 파일

os.chdir('/project/kcure-08/src/1. Data/1. Colon') # 현재 경로 변경(데이터 파일)
print(os.getcwd())
# print(os.listdir(os.getcwd()))

/project/kcure-08/src/2. Model/1. Colon
/project/kcure-08/src/1. Data/1. Colon


----

# 1. 데이터 불러오기

In [3]:
df_cs_cc = pd.read_csv('./PP_csv/cs_cc.csv') # 21897420 rows × 12 columns
# df_cc_rgst = pd.read_csv('./PP_csv/cs_cc_rgst.csv') # 21773 rows × 36 columns
# df_cc_g1e = pd.read_csv('./PP_csv/cs_cc_g1e.csv') # 52015 rows × 144 columns
# df_cc_ccr_cq_rst = pd.read_csv('./PP_csv/cs_cc_ccr_cq_rst.csv') # 32598 rows × 70 columns

----

# 2. 데이터 전처리
- [고찰필요] 동일 피험자 내 변수처리를 어떻게 할지
- Label 1은 대장암 진단종료 후 시점에서 재발(재발코드U99)가 발생한 지점에서 변수처리
- Label 0은 대장암 진단 시점에서 변수처리

In [ ]:
### 중요! ###
# before time of colon caner diagnosis
# agter time of colon cancer diagnosis
# time of cancer recurrence diagnosis

##### Label 1 - Table

In [19]:
df_cs_cc_rec_fin = df_cs_cc_rec[['SN_KEY']]
df_cs_cc_rec_fin['LABEL'] = 1
df_cs_cc_rec_fin['RES'] = list(df_cs_cc_rec['RVD_PL_CD'])
df_cs_cc_rec_fin['SOPR_Y_CNT'] = list(result['SOPR_Y_CNT'])

df_cs_cc_rec_fin['TIME_OF_NREC_END'] = pd.to_datetime(df_cs_cc_rec['TIME_OF_NREC_END'], format = '%Y-%m-%d', errors='raise')
df_cs_cc_rec_fin['TIME_OF_REC_START'] = pd.to_datetime(df_cs_cc_rec['TIME_OF_REC_START'], format = '%Y-%m-%d', errors='raise')
df_cs_cc_rec_fin['TIME_OF_REC'] = df_cs_cc_rec_fin['TIME_OF_REC_START'] - df_cs_cc_rec_fin['TIME_OF_NREC_END']

### 테이블 저장 ###
df_cs_cc_rec_fin.to_csv('./PP_csv/cs_cc_rec.csv', index = False) 

##### Label 0 - Table

In [20]:
df_cs_cc_nrec_fin = df_cs_cc_rec_id[['SN_KEY']]
df_cs_cc_nrec_fin['LABEL'] = 0
df_cs_cc_nrec_fin['RES'] = list(df_cs_cc_rec_id['RVD_PL_CD'])
df_cs_cc_nrec_fin['SOPR_Y_CNT'] = list(result_nrec['SOPR_Y_CNT'])

df_cs_cc_nrec_fin['TIME_OF_NREC_END'] = pd.to_datetime(df_cs_cc_rec_id['RECU_TO_DD'], format = '%Y-%m-%d', errors='raise')
df_cs_cc_nrec_fin['TIME_OF_REC_START'] = 'NaN'
df_cs_cc_nrec_fin['TIME_OF_REC'] = 'NaN'

### 테이블 저장 ###
df_cs_cc_nrec_fin.to_csv('./PP_csv/cs_cc_nrec.csv', index = False) 

----

## 2-1. [공통] Time
- 대장암 초진 전의 시간 (불필요) : 대장암 초진 종료 후 년도 기준으로
- 대장암 초진 종료 후 시간
- 재발 시간

---

### Label 1: 재발코드 U99 피험자 (재발시 다른 암 재발 포함)

In [ ]:
### 재발시점 진료 시작일자 찾기 ###
# 3092 rows × 12 columns
df_cs_cc_rec = df_cs_cc[((df_cs_cc['SICK_CD_NEW'] == 'U99')| (df_cs_cc['M_SICK_CD'] == 'U99') | (df_cs_cc['SUB_SICK_CD'] == 'U99')) & (df_cs_cc['SICK_TY_CD'] != 3.0)]

### 중복값 제거 및 첫번째 값 반환 ###
# 195 rows × 12 columns
df_cs_cc_rec = df_cs_cc_rec.drop_duplicates('SN_KEY') 

### 뒷단에 칼럼 추가 ###
# 195 rows × 13 columns
df_cs_cc_rec['TIME_OF_REC_START'] = df_cs_cc_rec['RECU_FR_DD']

### 재발 전 대장암 진단을 안받은 피험자 제외 ###
index_dp = df_cs_cc_rec[df_cs_cc_rec['SN_KEY'] == 'SN00042239'].index
df_cs_cc_rec = df_cs_cc_rec.drop(index_dp)
df_cs_cc_rec # 194 rows × 13 columns

######################################################################
## 같은 리스트 다른 요소 찾는 방법 ##
# list1 = list(df_cs_cc_tmp['SN_KEY'])
# list2 = list(df_cs_cc_rec['SN_KEY'])

# set1 = set(list1)
# set2 = set(list2)

# diff = set1.symmetric_difference(set2)
# result = list(diff)
# print(result)

######################################################################

In [ ]:
### 재발이전 대장암 첫 진료 종료일자 찾기 ###
# 8622521 rows × 12 columns
df_cs_cc_tmp = df_cs_cc.drop_duplicates('MID') 

### 재발코드가 없는 피험자 거르기 ###
cust_id = list(df_cs_cc_rec['SN_KEY'])
df_cs_cc_tmp = df_cs_cc_tmp[df_cs_cc_tmp['SN_KEY'].isin(cust_id)]

### 재발이전 대장암 진단 받은 피험자 찾기 ###
# 12760 rows × 12 columns
df_cs_cc_tmp = df_cs_cc_tmp[(df_cs_cc_tmp['SICK_CD_NEW'] == 'C18') | (df_cs_cc_tmp['SICK_CD_NEW'] == 'C19') | (df_cs_cc_tmp['SICK_CD_NEW'] == 'C20')| (df_cs_cc_tmp['M_SICK_CD'] == 'C18') | (df_cs_cc_tmp['M_SICK_CD'] == 'C19') | (df_cs_cc_tmp['M_SICK_CD'] == 'C20') | (df_cs_cc_tmp['SUB_SICK_CD'] == 'C18') | (df_cs_cc_tmp['SUB_SICK_CD'] == 'C19') | (df_cs_cc_tmp['SUB_SICK_CD'] == 'C20')]

### 첫 진료 종료일자 찾기 ###
# 194 rows × 14 columns
df_cs_cc_tmp_2 = df_cs_cc_tmp.drop_duplicates('SN_KEY') 
TIME_OF_NREC_END = list(df_cs_cc_tmp_2['RECU_TO_DD'])
df_cs_cc_rec['TIME_OF_NREC_END'] = TIME_OF_NREC_END
df_cs_cc_rec

----

##### RVD_PL_CD: 요양기관기호지역구분코드 --> 거주지
- 대도시(서울, 부산, 대구, 인천, 울산, 광주, 대전, 수원, 용인, 고양, 창원)
- 중소도시 (나머지)
- 재발시점에서 거주지

**대도시 : 0, 중소도시 : 1**

In [6]:
df_cs_cc_rec['RVD_PL_CD'] = df_cs_cc_rec['RVD_PL_CD'].astype('int32')

df_cs_cc_rec.loc[df_cs_cc_rec['RVD_PL_CD'] < 32, 'RVD_PL_CD'] = 0
df_cs_cc_rec.loc[df_cs_cc_rec['RVD_PL_CD'] >= 32, 'RVD_PL_CD'] = 1

df_cs_cc_rec['RVD_PL_CD'].value_counts()

0    124
1     70
Name: RVD_PL_CD, dtype: int64

---

##### SOPR_YN: 수술여부
- 대장암 첫 진료종료 시점에서의 수술여부

In [8]:
### 재발이전 대장암 진단 받은 피험자 추적 ###
# df_cs_cc_tmp

def count_9(group):
    return (group == 9).sum()

result = df_cs_cc_tmp.groupby('SN_KEY')['SOPR_YN'].agg(count_9).reset_index(name='SOPR_Y_CNT')
print(result['SOPR_Y_CNT'].value_counts())

4     35
3     35
2     29
6     19
5     18
7     13
1     11
8      9
11     4
13     3
12     3
9      3
0      2
18     2
14     2
10     2
30     1
25     1
15     1
16     1
Name: SOPR_Y_CNT, dtype: int64


----

### Label 0: 재발코드 U99 아닌 피험자

In [9]:
### 재발코드 피험자 거르기 ###
# 21702012 rows × 12 columns
cust_id = list(df_cs_cc_rec['SN_KEY'])
df_cs_cc_nrec = df_cs_cc[~df_cs_cc['SN_KEY'].isin(cust_id)]

In [ ]:
# 상병코드가 대장암 관련 행수 불러오기 #
# 3677039 rows × 12 columns
df_cs_cc_nrec = df_cs_cc_nrec[(df_cs_cc_nrec['SICK_TY_CD'] != 3.0)]
df_cs_cc_nrec = df_cs_cc_nrec[(df_cs_cc_nrec['SICK_CD_NEW'] == 'C18') | (df_cs_cc_nrec['SICK_CD_NEW'] == 'C19') | (df_cs_cc_nrec['SICK_CD_NEW'] == 'C20')| (df_cs_cc_nrec['M_SICK_CD'] == 'C18') | (df_cs_cc_nrec['M_SICK_CD'] == 'C19') | (df_cs_cc_nrec['M_SICK_CD'] == 'C20') | (df_cs_cc_nrec['SUB_SICK_CD'] == 'C18') | (df_cs_cc_nrec['SUB_SICK_CD'] == 'C19') | (df_cs_cc_nrec['SUB_SICK_CD'] == 'C20')]

### SN_KEY 중복값 제거 ###
# 21460 rows × 12 columns
df_cs_cc_rec_id = df_cs_cc_nrec.drop_duplicates('SN_KEY') 
df_cs_cc_rec_id # 21547 rows × 12 columns

----

##### RVD_PL_CD: 요양기관기호지역구분코드 --> 요양기관소재지
- 대도시(서울, 부산, 대구, 인천, 울산, 광주, 대전, 수원, 용인, 고양, 창원)
- 중소도시 (나머지)
- 재발시점에서 요양기관소재지

**대도시 : 0, 중소도시 : 1**

In [12]:
df_cs_cc_rec_id['RVD_PL_CD'] = df_cs_cc_rec_id['RVD_PL_CD'].astype('int32')

df_cs_cc_rec_id.loc[df_cs_cc_rec_id['RVD_PL_CD'] < 32, 'RVD_PL_CD'] = 0
df_cs_cc_rec_id.loc[df_cs_cc_rec_id['RVD_PL_CD'] >= 32, 'RVD_PL_CD'] = 1

df_cs_cc_rec_id['RVD_PL_CD'].value_counts()

0    16114
1     5346
Name: RVD_PL_CD, dtype: int64

----

##### SOPR_YN: 수술여부
- 대장암 첫 진료 시점에서의 수술여부

In [ ]:
### 명세서 중복제거 필요 ###
df_cs_cc_rec_mid = df_cs_cc_nrec.drop_duplicates('MID')
df_cs_cc_rec_mid

In [17]:
def count_9(group):
    return (group == 9).sum()

result_nrec = df_cs_cc_rec_mid.groupby('SN_KEY')['SOPR_YN'].agg(count_9).reset_index(name='SOPR_Y_CNT')
print(result_nrec['SOPR_Y_CNT'].value_counts())

1     5838
2     5654
3     3922
4     2263
0     1208
5     1186
6      614
7      280
8      181
9      100
10      62
11      34
12      26
13      17
15      15
14      13
16      11
21       5
17       5
24       3
18       3
22       3
19       2
34       1
50       1
33       1
51       1
27       1
55       1
26       1
32       1
29       1
35       1
30       1
42       1
20       1
31       1
45       1
Name: SOPR_Y_CNT, dtype: int64


In [18]:
result_nrec.describe()

,SOPR_Y_CNT
count,21460.000000
mean,2.560857
std,2.153241
min,0.000000
25%,1.000000
50%,2.000000
75%,3.000000
max,55.000000


----